# Exploring the Draft Model

This notebook contains our first dive into the _Vibrio natriegens_

In [ ]:
import cobra

In [ ]:
a = "string"

In [ ]:
b = 3

In [ ]:
c = 3.1

In [ ]:
d = ["apple", "citron", "banana"]

In [ ]:
e = set(d)

In [ ]:
f = dict([('key','value')])

In [ ]:
f['key']

In [ ]:
f = {'key':'value'}

In [ ]:
f['key']

In [ ]:
f = dict()

In [ ]:
if 3 < b < 4:
    print( "yeah this works")
elif b == 3:
    print( "blergh")
else:
    pass

In [ ]:
for fruit in d:
    print(fruit)

In [ ]:
d.append('melon')

In [ ]:
whos

## Creating a Model 

In [15]:
from __future__ import print_function

from cobra import  Model, Reaction , Metabolite

model=Model("example_model")


## Creating a reaction

In [17]:
reaction = Reaction('3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

## Creating metabolites

In [16]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    '3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite('co2_c', formula='CO2', name='CO2', compartment='c')
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite('h_c', formula='H', name='H', compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

# Adding metabolites to reaction

In [20]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

reaction.reaction  # This gives a string representation of the reaction

'ddcaACP_c + h_c + malACP_c --> 3omrsACP_c + ACP_c + co2_c'

# Assigning gene reaction rule string 

In [21]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction.genes

frozenset({<Gene STM1197 at 0x2b6266b50b8>, <Gene STM2378 at 0x2b6266b5d30>})

## Adding the reaction to the model

In [22]:
model.add_reactions([reaction])

# Now there are things in the model
print('%i reaction' % len(model.reactions))
print('%i metabolites' % len(model.metabolites))
print('%i genes' % len(model.genes))

1 reaction
6 metabolites
2 genes


## The contents so far

In [23]:
# Iterate through the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
3OAS140 : ddcaACP_c + h_c + malACP_c --> 3omrsACP_c + ACP_c + co2_c

Metabolites
-----------
 malACP_c : C14H22N2O10PRS
      h_c : H
ddcaACP_c : C23H43N2O8PRS
    co2_c : CO2
    ACP_c : C11H21N2O7PRS
3omrsACP_c : C25H45N2O9PRS

Genes
-----
STM1197 is associated with reactions: {3OAS140}
STM2378 is associated with reactions: {3OAS140}


### Setting the objective of the model

In [24]:
model.objective = '3OAS140'


In [25]:
print(model.objective.expression)
print(model.objective.direction)

-1.0*3OAS140_reverse_65ddc + 1.0*3OAS140
max


## Read/Write SBML

In [38]:

import cobra.test
import os
from os.path import join

data_dir = cobra.test.data_dir

print("mini test files: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("mini")))

cobra.io.read_sbml_model(join(data_dir, "mini_fbc2.xml"))

cobra.io.write_sbml_model(textbook_model, "test_fbc2.xml")

mini test files: 
mini.json, mini.mat, mini.pickle, mini.yml, mini_cobra.xml, mini_fbc1.xml, mini_fbc2.xml, mini_fbc2.xml.bz2, mini_fbc2.xml.gz


## Running FBA

In [52]:
solution = model.optimize()
print(solution)

solution.status



<Solution 0.000 at 0x2b62791c160>


'optimal'

In [54]:
model.summary 

<bound method Model.summary of <Model example_model at 0x2b6266332b0>>

In [56]:
model.optimize().objective_value



0.0

In [57]:
model.slim_optimize()

0.0

In [58]:
model.metabolites.ddcaACP_c.summary()

PRODUCING REACTIONS -- Dodecanoyl-ACP-n-C120ACP (ddcaACP_c)
-----------------------------------------------------------
%    FLUX    RXN ID    REACTION
---  ------  --------  ----------


CONSUMING REACTIONS -- Dodecanoyl-ACP-n-C120ACP (ddcaACP_c)
-----------------------------------------------------------
%    FLUX    RXN ID    REACTION
---  ------  --------  ----------



In [42]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction 3OAS140 at 0x2b626702ef0>: 1.0}

## Running FVA

In [48]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)



,minimum,maximum
3OAS140,0.0,0.0


In [49]:

from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(model, model.reactions[:10])

cobra.flux_analysis.flux_variability_analysis(
    model, model.reactions[:10], fraction_of_optimum=0.9)


,minimum,maximum
3OAS140,0.0,0.0


In [50]:
model.optimize()
model.summary(fva=0.95)

AttributeError: 'DataFrame' object has no attribute 'flux'

In [60]:
pfba_solution = cobra.flux_analysis.pfba(model) 


## Simulation deletions 

In [62]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

## Production envelopes

In [66]:
import cobra.test
from cobra.flux_analysis import production_envelope

## Flux sampling

In [69]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample

model = create_test_model("textbook")
s = sample(model, 100)
s.head()

,ACALD,ACALDt,ACKr,ACONTa,ACONTb,ACt2r,ADK1,AKGDH,AKGt2r,ALCD2x,ATPM,ATPS4r,Biomass_Ecoli_core,CO2t,CS,CYTBD,D_LACt2,ENO,ETOHt2r,EX_ac_e,EX_acald_e,EX_akg_e,EX_co2_e,EX_etoh_e,EX_for_e,EX_fru_e,EX_fum_e,EX_glc__D_e,EX_gln__L_e,EX_glu__L_e,EX_h_e,EX_h2o_e,EX_lac__D_e,EX_mal__L_e,EX_nh4_e,EX_o2_e,EX_pi_e,EX_pyr_e,EX_succ_e,FBA,FBP,FORt2,FORti,FRD7,FRUpts2,FUM,FUMt2_2,G6PDH2r,GAPD,GLCpts,GLNS,GLNabc,GLUDy,GLUN,GLUSy,GLUt2r,GND,H2Ot,ICDHyr,ICL,LDH_D,MALS,MALt2_2,MDH,ME1,ME2,NADH16,NADTRHD,NH4t,O2t,PDH,PFK,PFL,PGI,PGK,PGL,PGM,PIt2r,PPC,PPCK,PPS,PTAr,PYK,PYRt2,RPE,RPI,SUCCt2_2,SUCCt3,SUCDi,SUCOAS,TALA,THD2,TKT1,TKT2,TPI
0,-2.339220,-0.075365,-1.257018,4.328305,4.328305,-1.257018,5.621883,3.941197,-0.047951,-2.263855,12.028398,36.150334,0.021794,-37.492723,4.328305,73.745910,-0.799280,9.346173,-2.263855,1.257018,0.075365,0.047951,37.492723,2.263855,2.960814,0.0,0.0,-8.730114,2.467598e-19,0.008138,5.997253,35.848879,0.799280,0.0,-0.126976,-36.872955,-0.080174,0.267379,0.081697,0.684920,8.432166,4.050813,7.011627,704.622053,0.0,4.167005,0.0,24.070545,9.378777,8.730114,20.543396,-1.342498e-16,2.287337,18.129083,2.408740,-0.008138,24.070545,-35.848879,4.020800,0.307505,-0.799280,0.307505,0.0,3.201951,0.735984,0.536575,69.578905,54.557548,0.126976,36.872955,5.352914,9.117086,2.960814,-15.344899,-9.378777,24.070545,-9.346173,0.080174,2.717808,1.552515,5.621883,1.257018,5.061337,-0.267379,16.031365,-8.039181,67.177350,67.259047,708.789058,-3.941197,8.019616,2.264417,8.019616,8.011749,0.684920
1,-1.018999,-0.854861,-0.003160,11.224466,11.224466,-0.003160,5.140589,0.126376,-1.121967,-0.164137,20.712836,36.591950,0.040933,-24.439977,11.224466,62.869114,-0.493612,16.962241,-0.164137,0.003160,0.854861,1.121967,24.439977,0.164137,10.330875,0.0,0.0,-9.380444,8.519897e-18,0.168194,18.975147,29.678269,0.493612,0.0,-0.391392,-31.434557,-0.150579,0.301806,2.222132,7.711128,6.906315,32.310116,42.640991,20.107910,0.0,7.668011,0.0,4.885801,17.023476,9.380444,3.886621,2.610869e-15,0.316328,3.178901,0.697254,-0.168194,4.885801,-29.678269,1.460700,9.763766,-0.493612,9.763766,0.0,4.976049,7.020808,5.434920,55.201103,54.280142,0.391392,31.434557,11.832923,14.617443,10.330875,4.486251,-17.023476,4.885801,-16.962241,0.150579,14.499838,8.178287,5.140589,0.003160,6.379586,-0.301806,3.227778,-1.658023,5.504804,7.726936,27.775921,-0.126376,1.621278,38.527112,1.621278,1.606501,7.711128
2,-0.312667,-0.007366,-0.474177,12.723948,12.723948,-0.474177,0.429088,4.881546,-0.231272,-0.305301,8.951302,25.429741,0.170662,-33.043805,12.723948,73.767341,-0.108749,17.663941,-0.305301,0.474177,0.007366,0.231272,33.043805,0.305301,6.117324,0.0,0.0,-9.621544,3.517393e-17,0.197071,14.413578,36.753721,0.108749,0.0,-1.127654,-36.883671,-0.627813,0.397556,1.517808,8.581619,5.467000,58.198393,64.315718,29.398214,0.0,10.593671,0.0,2.610503,17.919251,9.621544,8.867150,2.847220e-15,3.205245,4.534251,4.289262,-0.197071,2.610503,-36.753721,5.494016,7.229932,-0.108749,7.229932,0.0,9.562382,2.233300,6.027921,63.173670,60.380567,1.127654,36.883671,15.263006,14.048620,6.117324,6.976055,-17.919251,2.610503,-17.663941,0.627813,14.155095,10.688608,0.429088,0.474177,4.916408,-0.397556,1.617664,-0.992839,22.801087,24.318895,39.991885,-4.881546,0.839636,46.945001,0.839636,0.778028,8.581619
3,-1.331966,-1.187502,-1.002816,11.118849,11.118849,-1.002816,7.851633,5.764800,-0.921708,-0.144464,9.732756,56.277470,0.003241,-31.428232,11.118849,72.439241,-0.857898,17.779118,-0.144464,1.002816,1.187502,0.921708,31.428232,0.144464,6.709703,0.0,0.0,-9.880378,2.759159e-17,0.325387,14.280906,35.989764,0.857898,0.0,-0.343060,-36.219620,-0.011924,1.225110,0.963085,7.908981,1.578388,6.695836,13.405539,136.725042,0.0,8.905172,0.0,5.904520,17.783967,9.880378,9.165061,2.054796e-15,8.481689,0.340311,8.823921,-0.325387,5.904520,-35.989764,7.015392,4.103457,-0.857898,4.103457,0.0,-0.847648,9.071580,4.784697,63.534069,67.877450,0.343060,36.219620,10.859532,9.487369,6.709703,3.975194,-17.783967,5.904520,-17.779118,0.011924,52

In [70]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

One process:
Wall time: 17.6 s
Two processes:
Wall time: 1min 8s


In [71]:
s = sample(model, 100, method="achr")

### Advanced usage

In [72]:
from cobra.flux_analysis.sampling import OptGPSampler, ACHRSampler


In [78]:
achr = ACHRSampler(model, thinning=10)

optgp = OptGPSampler(model, processes=4)

### Print out all output solution

In [82]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

### Sampling and validation

In [84]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
achr.validate(np.atleast_2d(bad)) 

s1 = achr.sample(100)

s2 = optgp.sample(100)

achr.validate(s1)

s1_valid = s1[achr.validate(s1) == "v"]
len(s1_valid)

array(['le'], dtype='<U3')

array(['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v'], dtype='<U3')

100

### Batch sampling

In [85]:
counts = [np.mean(s.Biomass_Ecoli_core > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(
    np.mean(counts) * 100.0, np.std(counts) * 100.0))

Usually 10.60% +- 0.80% grow...


### Adding constraints

In [87]:
co = model.problem.Constraint(model.reactions.Biomass_Ecoli_core.flux_expression, lb=0.1)
model.add_cons_vars([co])

In [11]:
print('original objective: ', model.objective.expression)
with model:
    model.objective = 'ATPM'
    print('print objective in first context:', model.objective.expression)
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)
    

original objective:  -1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
print objective in first context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM


In [10]:
model = cobra.test.create_test_model('textbook')
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


In [88]:
s = sample(model, 10)
print(s.Biomass_Ecoli_core)

0    0.104775
1    0.134553
2    0.137393
3    0.105585
4    0.101469
5    0.103592
6    0.118790
7    0.131271
8    0.162825
9    0.113454
Name: Biomass_Ecoli_core, dtype: float64


### Loopless FBA

In [ ]:
%matplotlib inline
import plot_helper

import cobra.test
from cobra import Reaction, Metabolite, Model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

In [ ]:
salmonella = cobra.test.create_test_model('salmonella')
nominal = salmonella.optimize()
loopless = loopless_solution(salmonella)

import pandas
df = pandas.DataFrame(dict(loopless=loopless.fluxes, nominal=nominal.fluxes))

df.plot.scatter(x='loopless', y='nominal')